In [2]:
import json
import os
import re
from collections import defaultdict

import sys

# 필요한 경로 추가
base_path = '/home/seonghee_hong/legal_llm/dps'
sys.path.append(f'{base_path}/dps/spark')
sys.path.append(f'{base_path}/dps/spark/jobs')

from jobs.korean_job import *
from jobs.dedup_job import *
from prep.dedup_prep import *

In [3]:
# 테스트를 위한 샘플 텍스트
sample_text = "이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다."

# make_compat() 함수 테스트
compat_text = make_compat(sample_text)
print("After make_compat:", compat_text)

# bad_words_filter() 함수 테스트
bad_words_result = bad_words_filter(compat_text)
print("Bad words filter result:", bad_words_result)

# doc_len_filter() 함수 테스트
doc_len_result = doc_len_filter(compat_text, 10, 500)  # 예시로 최소 길이 10, 최대 길이 500 설정
print("Document length filter result:", doc_len_result)

# mean_word_len_filter() 함수 테스트
mean_word_len_result = mean_word_len_filter(compat_text, 2, 10)  # 예시로 최소 평균 단어 길이 2, 최대 10 설정
print("Mean word length filter result:", mean_word_len_result)

# symbol_to_word_ratio_filter() 함수 테스트
symbol_to_word_ratio_result = symbol_to_word_ratio_filter(compat_text, 0.1)  # 예시로 비율 0.1 설정
print("Symbol to word ratio filter result:", symbol_to_word_ratio_result)

# bullet_ellipsis_filter() 함수 테스트
bullet_ellipsis_result = bullet_ellipsis_filter(compat_text, 0.05, 0.05)  # 예시로 비율 각각 0.05 설정
print("Bullet and ellipsis filter result:", bullet_ellipsis_result)

# korean_word_ratio_filter() 함수 테스트
korean_word_ratio_result = korean_word_ratio_filter(compat_text, 0.5)  # 예시로 한국어 비율 0.5 설정
print("Korean word ratio filter result:", korean_word_ratio_result)

# preprocess_text() 함수 테스트
preprocessed_text = preprocess_text(compat_text)
print("Preprocessed text:", preprocessed_text)

After make_compat: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.
Bad words filter result: True
Document length filter result: True
Mean word length filter result: True
Symbol to word ratio filter result: True
Bullet and ellipsis filter result: True
Korean word ratio filter result: True
Preprocessed text: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.


In [4]:
### Load Sample Data
import json
import random

# Fix Random Seed
random.seed(42)

with open('/data/llmlaw/FIRST_DEDUPLICATION/Domain/CASE-LAW/deduplication.jsonl', 'r') as f:
    case_data_og = [json.loads(line) for line in f]

with open('/data/llmlaw/FIRST_DEDUPLICATION/Domain/NON-CASE-LAW/deduplication.jsonl', 'r') as f:
    non_case_data_og = [json.loads(line) for line in f]

print("Total CASE data after first deduplication:", len(case_data_og))
print("Total NON-CASE data after first deduplication:", len(non_case_data_og))

# Random Sample
case_sample = random.sample(case_data_og, 20000)
non_case_sample = random.sample(non_case_data_og, 20000)

Total CASE data after first deduplication: 192730
Total NON-CASE data after first deduplication: 231179


### DPS 상에서 코드 전처리 파이프라인과 동일하게 Chaining 방식으로 테스트

In [5]:
# Text만 추출
case_text = [case['text'] for case in case_sample]
non_case_text = [non_case['text'] for non_case in non_case_sample]

In [ ]:
# 테스트를 위한 샘플 텍스트
sample_text = "이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다."

# make_compat() 함수 테스트
compat_text = make_compat(sample_text)
print("After make_compat:", compat_text)

# bad_words_filter() 함수 테스트
"""Drop text that contains bad words"""
if bad_words_filter(compat_text):
    print("Passed bad words filter")
else:
    print("Failed bad words filter")
    # 필터에 실패한 경우, 테스트 중단
    compat_text = None

# doc_len_filter() 함수 테스트, 통과 시 계속 진행
"""Filter any doc that does not contain between min_doc_len and max_doc_len words"""
if compat_text and doc_len_filter(compat_text, min_doc_len=36, max_doc_len=299675):  # 최소 길이 10, 최대 길이 500
    print("Passed document length filter")
else:
    print("Failed document length filter")
    compat_text = None

# mean_word_len_filter() 함수 테스트
"""Filter any doc whose mean word length is outside the range of min_word_len to max_word_len characters"""
if compat_text and mean_word_len_filter(compat_text, 2, 10):  # 최소 평균 단어 길이 2, 최대 10
    print("Passed mean word length filter")
else:
    print("Failed mean word length filter")
    compat_text = None

# symbol_to_word_ratio_filter() 함수 테스트 # TODO [08.13] In Progress
"""Filter any doc with a symbol-to-word ratio greater than symbol_to_word_ratio for either the hash symbol or the
ellipsis"""
if compat_text and symbol_to_word_ratio_filter(compat_text, symbol_to_word_ratio=0.1):  # 비율 0.1 설정
    print("Passed symbol to word ratio filter")
else:
    print("Failed symbol to word ratio filter")
    compat_text = None

# bullet_ellipsis_filter() 함수 테스트
"""Filter any doc with more than bullet_point_ratio of lines starting with a bullet point, or more than
ellipsis_ratio ending with an ellipsis"""
if compat_text and bullet_ellipsis_filter(compat_text, bullet_point_ratio=0.05, ellipsis_ratio=0.05):  # 비율 각각 0.05 설정
    print("Passed bullet and ellipsis filter")
else:
    print("Failed bullet and ellipsis filter")
    compat_text = None

# korean_word_ratio_filter() 함수 테스트
"""Filter any doc with a Korean word ratio less than korean_word_ratio"""
if compat_text and korean_word_ratio_filter(compat_text, korean_word_ratio=0.5):  # 한국어 비율 0.5 설정
    print("Passed Korean word ratio filter")
else:
    print("Failed Korean word ratio filter")
    compat_text = None

# preprocess_text() 함수 테스트
"""아래 함수들을 실행함
    processing_function_list = [
        process_html_and_uri_text,
        reduce_emoticon,
        remove_whitespace,
        replace_email_and_url,
        replace_korean_pii,
        spam_words_filter,
        remove_html_tags,
        remove_repeated_text,
    ]
"""

if compat_text:
    preprocessed_text = preprocess_text(compat_text)
    print("Preprocessed text:", preprocessed_text)
else:
    print("Text did not pass the filters, preprocessing skipped.")
